In [48]:
import sys
import pickle
from network_model import *
from SALib.sample import sobol, saltelli
from tqdm import tqdm


import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({"font.size" : 15,
                     "figure.dpi" : 100, 
                     "grid.alpha" : 0.3, 
                     "axes.grid": True, 
                     "axes.axisbelow" : True,
                     "figure.figsize":(8,6),
                     "mathtext.fontset":"cm",
                     "xtick.labelsize": 14,
                     "ytick.labelsize": 14,
                     "axes.labelsize": 16, 
                     "legend.fontsize": 13.5})
USE_TEX = False
if USE_TEX:
    plt.rc("text", usetex=True)
    plt.rc("text.latex", preamble=r"""
     \usepackage{times}
     \usepackage{mathptmx}""")
else:
    plt.rc("text", usetex=False)
plt.rc("font", family="serif")

In [42]:
def sda_prob(d, a, b):
    return 1 / (1 + ((1/b)*d)**a)


def sda_graph(P):
    G = nx.Graph()
    # create nodes
    for i in range(1250):
        G.add_node(i)
    coin_flips = np.random.uniform(size=780625) # 780625 = number of pairwise distances for 1250 agents
    k = 0
    # add edges probabilistically
    for i in range(len(P)):
        for j in range(i+1, len(P)):
            if P[i][j] > coin_flips[k]:
                G.add_edge(i,j)
            k += 1  
    return G

In [66]:
# problem definition
PROBLEM = {
    "num_vars" : 7,
    "names"    : ["project_cost",
                  "gain_right",
                  "alpha_beta",
                  "prob_left",
                  "init_w_scale",
                  "risk_scale",
                  "poisson_scale"],
    "bounds"   : [[0.01, 2.00],
                  [1.70, 2.30],
                  [0.70, 0.80],
                  [0.30, 0.45],
                  [0.01, 0.15],
                  [5.00, 20.0],
                  [8.00, 20.0]]
}

# generate Saltelli samples
NUM_SAMPLES = 1024
X = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)

unique_w_scales = sorted(list(set(X[:,4])))

/var/folders/bj/4q98hxqx4vq6wwm8qqllvxmr0000gn/T/ipykernel_18063/173000835.py:22: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  X = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)


In [67]:
# set random seed
random.seed(123)
np.random.seed(123)

for scale in tqdm(unique_w_scales):

    # create random initial wealth distribution
    W = np.random.normal(1, scale, size=1250)

    # compute pairwise distances
    pairwise_distances = []
    for i in range(len(W)):
        for j in range(i+1, len(W)):
            pairwise_distances.append(np.abs(W[i]-W[j]))
    mean_dist = np.mean(pairwise_distances)

    # SDA graph parameters
    a = 16
    b = mean_dist/15
    prob_attach = np.zeros((1250,1250))
    k = 0
    for i in range(len(W)):
        for j in range(i+1, len(W)):
            p = sda_prob(pairwise_distances[k], a=a, b=b)
            prob_attach[i][j] = p
            prob_attach[j][i] = p
            k+=1
    G = sda_graph(prob_attach)

    # ensure that graph is connected
    connected_components = list(nx.connected_components(G))
    largest_component = list(connected_components[0])
    largest_component_wealths = W[np.array(largest_component)]
    for i in range(1, len(connected_components)):
        comp = list(connected_components[i])
        node = np.random.choice(comp)
        closest_largest_component_node = (np.abs(largest_component_wealths - W[node])).argmin()
        G.add_edge(node, largest_component[closest_largest_component_node])
    assert nx.is_connected(G)

    # extract communities and construct community membership dictionary
    communities = get_communities(G)
    community_membership = get_community_membership(G, communities)

    # store results
    data_tuple = (G, communities, community_membership, W)
    with open(f"./sda_graphs/{scale}.pickle", "wb") as f:
        pickle.dump(data_tuple, f)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [23:42<00:00,  1.39s/it]
